<header style="padding:10px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

# Vantage Analytic Library Demo Notebook 2 

</header>

## Hypothesis Tests

Statistical tests provide a means of testing whether the outcome of an experiment could have been accidental.  Numerous tests are provided in-database, all of the hypothesis testing variety.  These include the following 18 tests, categorized below, that are currently available in the Vantage Analytic Library XSP release 2.0.

### Parametric Tests

- Two Sample T-Test for Equal Means
    - T Paired
    - T Unpaired
    - T Unpaired with Indicator
- F-Test – N-Way
- F-Test/Analysis of Variance (Two Way Unequal Sample Size)

### Binomial Tests

- Binomial/Ztest
- Binomial Sign Test

### Tests based on Contingency

- Chi Square Test
- Median Test

### Kolmogorov/Smirnoff Tests
    
- Kolmogorov/Smirnoff Test (One Sample)
- Lilliefors Test
- Shapiro-Wilk Test
- D’Agostino and Pearson Test
- Smirnov Test

### Rank Tests

- Mann-Whitney/Kruskal-Wallis Test
- Mann-Whitney/Kruskal-Wallis Independent Tests
- Wilcoxon Signed Ranks Test
- Frieman Test with Kendall’s Coefficient of Concordance & Spearman’s Rho

In order to execute the Statistical Tests described in this book, the Statistical Test Metadata tables must be loaded into a database on the system to be analyzed.  These tables have been loaded into the TRNG_XSP database along with the other VAL XSP and Table Operators.  The Statistical Test functions all provide a parameter called statsdatabase that can be used to specify the database in which these tables are installed.

## For access to the Vantage Analytic Library XSP on Transcend, please open a Service Hub incident @

>- https://teradataservicehub.service-now.com/sp?id=index
>- and begin the incident description with “This Incident is directed to the ‘IDW DBA Admin team.”
>- Rest of the description: “Please grant the following users the role Training_TD_Warehouse_Miner_Exec_Role in tdprd2.
>- Qlid_1
>- Qlid_2
>- Qlid_n “

## Vantage Analytic Library - Call Structure

call ${XSPDB}.td_analyze('\<function name\>','\<database=database name\>;\<tablename=table name\>;\<columns=column1, column2\>,'\<param1\>=\<value1\>;\<param2\>=\<value2\>...');

---

### Accessing the Data in DemoNow
<p style = 'font-size:16px;font-family:Arial'>The Vantage Analytic Library demos will work either with foreign tables accessed from Cloud Storage via NOS or you may import the tables to your machine. You only need to import them once for all of this series of notebooks. If you import data for multiple demos, you may need to use the Data Dictionary "Manage Your Space" routine to cleanup tables you no longer need. 
    
<p style = 'font-size:16px;font-family:Arial'>Use the link below to access the 2 options for using data from the data dictionary notebook:

[Click Here to get data for this notebook](../Data_Dictionary/Data_Dictionary.ipynb#TRNG_XSP)

[Click Here to Manage Your Space](../Data_Dictionary/Data_Dictionary.ipynb#Manage_Your_Space)
    
In the instructions below, use "SystemName=local" and "QLID=demo_user"

### Accessing the Data in Transcend AppCenter
These notebooks are configured to run from within the Transcend AppCenter Jupyter instance.  You will sign on to Transcend Vantage systems (tdprd, tdprd2 or tdprd3 (AKA Vantage LIVE)) using your credentials.  First we setup variables that you will need to change as follows.

1) First, setup the variable SystemName for the system to connect to.  In the Transcend environment, this will be one of the following:

* NAME=Transcend-Production, USER=, HOST=tdprd.td.teradata.com, PROPS="logMech=LDAP,logmech=LDAP"
* NAME=Transcend-Production-AWS, USER=, HOST=tdprd2.td.teradata.com, PROPS="logMech=LDAP,logmech=LDAP"
* NAME=Vantage-LIVE, USER=, HOST=tdprd3.td.teradata.com, PROPS="logMech=LDAP,logmech=LDAP"


In [1]:
%var SystemName=local

2) Next change QLID below to be equal to your QuickLook ID for Transcend or demo_user for DemoNow.  Your data lab will be the result database for all VAL calls where any output tables or views are created.  

In [2]:
%var QLID=demo_user

3) This next variable has been set for you to change, ONLY if you have installed the Vantage Analytic Library outside of the Transcend environment; if you are using this on Transcend, keep it set to TRNG_XSP.  This is where the software, statistical test tables and demo data are installed on both tdprd and tdprd2.

In [3]:
%var XSPDB=TRNG_XSP

In [4]:
%var VALDB=val

---

Now, connect to the Transcend system you have specified in the variable "SystemName" above.

In [6]:
%connect ${SystemName}

Connected: 'local' connection activated for user 'demo_user'


Change focus to the database specified by the variable "XSPDB" above.

In [7]:
DATABASE ${XSPDB};

Success: 1 rows affected

### Demo data - Financial Customers/Accounts/Transactions

The following data has beenn put into the ${XSPDB} database on Transcend for the examples in the three different Jupyter Notebooks.  Its a simplistic ficticiouss dataset of banking customers (10K-ish rows), Accounts (20K-ish rows) and Transactions (1M-ish rows).  They are related to each other in the following ways:

![DemoDataModel](./img/DemoData.png)

In [8]:
SELECT * FROM ${XSPDB}.Customer SAMPLE 10;

,cust_id,income,age,years_with_bank,nbr_children,gender,marital_status,postal_code,state_code
1,21807232,19845.8,53,6,1,F,2,98186,WA
2,21808416,128531.0,44,7,5,M,2,32235,FL
3,23163758,19001.7,36,2,2,F,2,10161,NY
4,17715698,11453.0,31,8,4,F,2,70156,LA
5,16354740,5541.3,59,9,4,M,2,10126,NY
6,24535710,22312.8,62,0,2,F,2,10084,NY
7,25901332,6885.0,46,3,0,M,1,02189,MA
8,25902605,834.3,71,8,0,F,4,85046,AZ
9,13624980,9849.0,61,1,1,F,2,10007,NY
10,27249760,14166.0,77,9,0,F,2,90159,CA


In [9]:
%meta

Result Set ID: /home/jovyan/JupyterLabRoot/Teradata/Resultsets/2022.09.16_20.58.16.212_UTC
History ID:    313
Rows:          10 of 10
Parts:         2
Column Definitions:
    cust_id: INTEGER
    income: DECIMAL(15, 1)
    age: INTEGER
    years_with_bank: INTEGER
    nbr_children: INTEGER
    gender: VARCHAR(1)
    marital_status: VARCHAR(1)
    postal_code: VARCHAR(5)
    state_code: VARCHAR(2)


In [10]:
SELECT * FROM ${XSPDB}.Accounts SAMPLE 10;

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,starting_balance,ending_balance
1,000000001362904220,27258080,CK,Y,1987-03-23,1974.918,3279.348
2,000000001363209312,16358508,SV,N,1986-02-03,317.628,3809.190
3,000000001362742222,29980324,CK,Y,1991-12-01,2912.162,23320.206
4,456114321363385423,31357855,CC,Y,1988-03-15,845.470,90.000
5,000000001363298222,29992556,CK,N,1993-09-17,8206.253,1547.364
6,456114321363420419,25904980,CC,Y,1993-09-11,563.985,3134.720
7,456114321362740415,20441100,CC,Y,1994-05-29,4222.341,5721.588
8,456114321363093416,21809488,CC,Y,1986-05-30,788.578,1020.000
9,456114321363284414,19085976,CC,Y,1990-07-21,1166.116,3960.272
10,000000001363354212,16360248,CK,Y,1994-03-27,1254.312,102.690


In [11]:
%meta

Result Set ID: /home/jovyan/JupyterLabRoot/Teradata/Resultsets/2022.09.16_20.58.18.586_UTC
History ID:    314
Rows:          10 of 10
Parts:         2
Column Definitions:
    acct_nbr: VARCHAR(18)
    cust_id: INTEGER
    acct_type: VARCHAR(2)
    account_active: VARCHAR(1)
    acct_start_date: DATE
    starting_balance: DECIMAL(11, 3)
    ending_balance: DECIMAL(11, 3)


In [12]:
SELECT * FROM ${XSPDB}.Transactions SAMPLE 10;

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
1,1495,000000001362594223,-957.14,-957.14,.000,10168.84,1995-08-08,940,A,TR
2,910,000000001363017213,-185.51,-140.51,.000,380.45,1995-02-16,0,P,WD
3,380,000000001362926219,-20.00,58.00,.000,3673.58,1995-03-23,91532,A,WD
4,60,000000001362806320,3.59,.00,7.590,1440.97,1995-03-31,235959,,IN
5,110,000000001362796222,.32,9.00,.243,252.69,1995-05-31,235959,,IN
6,560,456114321363327414,-189.61,-43.61,.760,-2878.81,1995-11-10,110213,E,CG
7,532,000000001362512219,190.70,268.70,.000,1934.92,1995-02-23,211532,A,DP
8,200,000000001362875310,.00,.00,.000,449.10,1995-12-05,91018,V,IQ
9,162,000000001362698318,273.50,1094.00,.000,414.74,1995-01-11,91613,A,DP
10,608,000000001362562319,.00,78.00,.000,4203.43,1995-11-24,190451,A,IQ


In [13]:
%meta

Result Set ID: /home/jovyan/JupyterLabRoot/Teradata/Resultsets/2022.09.16_20.58.28.147_UTC
History ID:    315
Rows:          10 of 10
Parts:         2
Column Definitions:
    tran_id: INTEGER
    acct_nbr: VARCHAR(18)
    tran_amt: DECIMAL(9, 2)
    principal_amt: DECIMAL(15, 2)
    interest_amt: DECIMAL(11, 3)
    new_balance: DECIMAL(9, 2)
    tran_date: DATE
    tran_time: INTEGER
    channel: VARCHAR(1)
    tran_code: VARCHAR(2)


---

The following Analytic Data Set (ADS) was created by joining all three tables above:

In [14]:
CREATE TABLE ${QLID}.VAL_ADS AS (
    SELECT 
        T1.cust_id  AS cust_id
       ,MIN(T1.income) AS tot_income
       ,MIN(T1.age) AS tot_age
       ,MIN(T1.years_with_bank) AS tot_cust_years
       ,MIN(T1.nbr_children) AS tot_children
       ,CASE WHEN MIN(T1.marital_status) = 1 THEN 1 ELSE 0 END AS single_ind
       ,CASE WHEN MIN(T1.gender) = 'F' THEN 1 ELSE 0 END AS female_ind
       ,CASE WHEN MIN(T1.marital_status) = 2 THEN 1 ELSE 0 END AS married_ind
       ,CASE WHEN MIN(T1.marital_status) = 3 THEN 1 ELSE 0 END AS separated_ind
       ,MAX(CASE WHEN T1.state_code = 'CA' THEN 1 ELSE 0 END) AS ca_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'NY' THEN 1 ELSE 0 END) AS ny_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'TX' THEN 1 ELSE 0 END) AS tx_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'IL' THEN 1 ELSE 0 END) AS il_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'AZ' THEN 1 ELSE 0 END) AS az_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'OH' THEN 1 ELSE 0 END) AS oh_resident_ind
       ,MAX(CASE WHEN T2.acct_type = 'CK' THEN 1 ELSE 0 END) AS ck_acct_ind
       ,MAX(CASE WHEN T2.acct_type = 'SV' THEN 1 ELSE 0 END) AS sv_acct_ind
       ,MAX(CASE WHEN T2.acct_type = 'CC' THEN 1 ELSE 0 END) AS cc_acct_ind
       ,AVG(CASE WHEN T2.acct_type = 'CK' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS ck_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'SV' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS sv_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'CC' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS cc_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'CK' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS ck_avg_tran_amt
       ,AVG(CASE WHEN T2.acct_type = 'SV' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS sv_avg_tran_amt
       ,AVG(CASE WHEN T2.acct_type = 'CC' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS cc_avg_tran_amt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 1 THEN T3.tran_id ELSE NULL END) AS q1_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 2 THEN T3.tran_id ELSE NULL END) AS q2_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 3 THEN T3.tran_id ELSE NULL END) AS q3_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 4 THEN T3.tran_id ELSE NULL END) AS q4_trans_cnt
    FROM ${XSPDB}.Customer AS T1
        LEFT OUTER JOIN ${XSPDB}.Accounts AS T2
            ON T1.cust_id = T2.cust_id
        LEFT OUTER JOIN ${XSPDB}.Transactions AS T3
            ON T2.acct_nbr = T3.acct_nbr
GROUP BY T1.cust_id) WITH DATA UNIQUE PRIMARY INDEX (cust_id);

Success: 0 rows affected

In [15]:
SELECT * FROM ${QLID}.VAL_ADS SAMPLE 10;

,cust_id,tot_income,tot_age,tot_cust_years,tot_children,single_ind,female_ind,married_ind,separated_ind,ca_resident_ind,ny_resident_ind,tx_resident_ind,il_resident_ind,az_resident_ind,oh_resident_ind,ck_acct_ind,sv_acct_ind,cc_acct_ind,ck_avg_bal,sv_avg_bal,cc_avg_bal,ck_avg_tran_amt,sv_avg_tran_amt,cc_avg_tran_amt,q1_trans_cnt,q2_trans_cnt,q3_trans_cnt,q4_trans_cnt
1,28623357,.0,17,2,1,1,0,0,0,1,0,0,0,0,0,1,1,1,1502.064193548387,631.2735483870968,331.93548387096774,-8.908510752688173,5.897344086021505,-2.1603817204301077,113,26,24,23
2,16357044,714.0,21,6,3,1,1,0,0,0,0,0,0,0,0,1,0,1,339.7906717557252,0,539.7369160305343,-4.2223664122137405,0,-14.613053435114503,83,34,6,8
3,19087418,5945.8,56,6,2,0,1,1,0,0,0,0,0,0,0,1,1,1,2449.879775147929,153.5735147928994,1003.6131597633137,1.5108284023668639,0.11272189349112426,0.8392307692307692,114,15,16,24
4,19082518,11559.8,30,9,1,0,1,1,0,0,0,0,0,0,0,1,1,1,3224.138171641791,1652.9376865671643,305.97014925373134,-3.543358208955224,3.2572388059701494,-1.6597014925373135,104,8,4,18
5,23165050,12640.8,26,1,1,0,0,1,0,0,1,0,0,0,0,0,1,1,0,417.46275862068967,1075.5172413793102,0,49.72362068965517,8.96551724137931,3,9,8,38
6,27261580,50852.0,46,8,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,17725110,23907.0,34,11,4,0,1,1,0,0,0,0,0,0,0,1,0,1,19404.9186122449,0,3640.816326530612,14.850102040816326,0,-18.43166666666667,65,39,16,27
8,17719286,34273.2,37,4,4,0,0,1,0,0,0,0,0,0,0,1,1,1,1117.9458196721312,226.96839344262295,751.5504918032786,25.37016393442623,8.902622950819673,16.15770491803279,13,15,17,16
9,17719403,68304.6,31,8,5,0,0,0,0,1,0,0,0,0,0,1,0,0,8161.031,0,0,58.24540404040404,0,0,99,0,0,0
10,28617015,3727.5,57,7,2,0,1,1,0,0,1,0,0,0,0,1,1,1,1699.2483050847457,37.05858757062147,413.9635593220339,-8.099768361581921,1.6476666666666666,0.04414689265536723,115,19,21,22


## Statistical Tests

### Binomial Tests

#### Purpose
In a binomial test there are assumed to be n independent trials, each with two possible outcomes, each of equal probability.  You may choose to perform a binomial test, in which the sign of the difference between a first and second column is analyzed, or a sign test, in which the sign of a single column is analyzed.  In a binomial test, you may choose to use a probability different from the 0.5 default value, whereas in a sign test, the binomial probability is fixed at 0.5.

#### Required Parameters

- **binomialtest**

    The binomialtest parameter:
    - Is required
    - Must be the first parameter
    - Is always enclosed in single quotes


- **database**

    The database containing the table to analyze.

- **firstcolumn**

    The column to analyze. It must reside in the table indicated by the tablename parameter

- **tablename**

    The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.

#### Optional Parameters

- **binomialprobability**

    You may optionally change the binomial probability or allow it to default to 0.5.  This parameter may not be used for a sign test.

- **exactmatches**

    This parameter determines what category to place exact matches in.  If zero, exact match is discarded.  If positive, match is placed with values greater than or equal to zero.  If negative or parameter not specified (default), match is placed with values less than or equal to zero.  Note that the exactmatches parameter is not allowed if this is a sign test.

- **fallback**

    When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.

- **gensqlonly**

    When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.

- **groupby**

    One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.

- **multiset**

    When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.

- **outputdatabase**

    Specifies the name of the database to contain the analysis results table.

- **outputtablename**

    Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.

- **overwrite**

    When overwrite is set to true or not set, the output table is dropped before creating a new one.

- **secondcolumn**

    A second column is required when performing a binomial test, and not allowed when performing a sign test.

- **singletail**

    A single-tailed test may be requested by setting this parameter to true.  A two-tailed test may be requested by setting this parameter to false or by not specifying it at all (the default value).  If the binomial probability is not 0.5, singletail must be set to true.

- **statsdatabase**

    This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.

- **teststyle**

    The valid values are binomial and sign, representing a binomial or sign test.  If not specified, the default test style is binomial.

- **thresholdprobability**

    You may optionally change the threshold probability or allow it to default to 0.05.  Below this  "alpha" probability the null hypothesis is rejected

1.  Binomial test producing an unpersisted result set with default values for thresholdprobabilty (.05), binomialprobability (0.5), singletail (false) and exactmatches (negative).

In [16]:
call ${VALDB}.td_analyze('BinomialTest',
                         'database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          secondcolumn=ck_avg_bal;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,N,NPos,NNeg,BP,BinomialCallP_0.05
1,10458,6384,4074,2e-06,p


2.  Binomial test producing a result table and introducing a groupby variable gender with default values for other parameters.

In [17]:
call ${VALDB}.td_analyze('BinomialTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          secondcolumn=ck_avg_bal;
                          groupby=female_ind;
                          outputdatabase=${QLID};
                          outputtablename=_val_binomial_example2;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

In [18]:
SELECT * FROM ${QLID}._val_binomial_example2;

,female_ind,N,NPos,NNeg,BP,BinomialCallP_0.05
1,1,5852,3717,2135,2e-06,p
2,0,4606,2667,1939,2e-06,p


3.  Sign test producing an unpersisted result set with default values for thresholdprobabilty (.05) and singletail (false).

In [19]:
call ${VALDB}.td_analyze('BinomialTest',
                         'database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          teststyle=sign;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,N,NPos,NNeg,BP,BinomialCallP_0.05
1,10458,5894,4564,2e-06,p


4.  Sign test producing a result table and introducing a groupby variable gender with default values for other parameters.

In [20]:
call ${VALDB}.td_analyze('BinomialTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          groupby=female_ind;
                          outputdatabase=${QLID};
                          outputtablename=_val_binomial_example4;
                          teststyle=sign;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

In [21]:
SELECT * FROM ${QLID}._val_binomial_example4;

,female_ind,N,NPos,NNeg,BP,BinomialCallP_0.05
1,1,5852,3514,2338,2e-06,p
2,0,4606,2380,2226,0.023268256,p


---

### Kolmogorov-Smirnoff Tests

#### Purpose

Kolmogorov-Smirnov Goodness-of-Fit Test is used to decide if a sample comes from a population with a specific distribution.  An attractive feature of this test is that the distribution of the K-S test statistic itself does not depend on the underlying cumulative distribution function being tested. Another advantage is that it is an exact test.

#### Required Parameters

- **columnofinterest**

    The numeric column that is tested to have a normal distribution. It must reside in the table indicated by the tablename parameter

- **database**

    The database containing the table to analyze.

- **kstest**

    The kstest parameter:
    - Is required
    - Must be the first parameter
    - Is always enclosed in single quotes


- **tablename**

    The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.

#### Optional Parameters

- **columns**

    Used only by the Smirnov test, this parameter represents a categorical variable with two values that indicate the distribution to which the columnofinterest belongs. It must reside in the table indicated by the tablename parameter.

- **fallback**

    When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.

- **gensqlonly**

    When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.  This parameter is not allowed when using the Smirnov test.

- **groupby**
    
    One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.

- **multiset**

    When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.

- **outputdatabase**

    Specifies the name of the database to contain the analysis results table.

- **outputtablename**

    Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.

- **overwrite**

    When overwrite is set to true or not set, the output table is dropped before creating a new one.

- **statsdatabase**

    This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.

- **teststyle**

    The valid values are ks, l, sw, p and s, representing a Kolmogorov-Smirnov, Lilliefors, Shapiro-Wilk, D’Agostino and Pearson or Smirnov test.  If not specified, the default test style is Kolmogorov-Smirnov (ks).

- **thresholdprobability**

    You may optionally change the threshold probability or allow it to default to 0.05.  Below this  "alpha" probability the null hypothesis is rejected.

1.  Kolmogorov-Smirnov test with group-by option.

In [22]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=KS;
                          groupby=tot_cust_years;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,tot_cust_years,Klm,M,KlmPValue,KlmPText,KlmCallP_5E-2
1,8,0.167601957,1014,0.01,<0.01,p
2,11,0.147381487,374,0.01,<0.01,p
3,12,0.145172899,182,0.01,<0.01,p
4,0,0.15574236700000005,176,0.01,<0.01,p
5,3,0.169747933,1160,0.01,<0.01,p
6,9,0.156509661,845,0.01,<0.01,p
7,4,0.17757486300000003,1137,0.01,<0.01,p
8,14,0.11384552699999997,39,0.2,>0.20,a
9,1,0.168141482,526,0.01,<0.01,p
10,10,0.186458308,613,0.01,<0.01,p


2.  Lilliefors test with group-by option.

In [23]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=L;
                          groupby=tot_cust_years;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,tot_cust_years,Lilliefors,M,LillieforsPValue,LillieforsPText,LillieforsCallP_5E-2
1,8,0.16437102548323468,1014,0.01,<0.01,p
2,11,0.14539964207486628,374,0.01,<0.01,p
3,12,0.14397700704395605,182,0.01,<0.01,p
4,0,0.15455723972727275,176,0.01,<0.01,p
5,3,0.1652836109310345,1160,0.01,<0.01,p
6,9,0.15286021289940832,845,0.01,<0.01,p
7,4,0.17220877782761657,1137,0.01,<0.01,p
8,14,0.11152678682051276,39,0.2,>0.20,a
9,1,0.16513402253992393,526,0.01,<0.01,p
10,10,0.1838866707161501,613,0.01,<0.01,p


3.  Shapiro-Wilk test with group-by option

In [25]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=SW;
                          groupby=tot_cust_years;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,tot_cust_years,Shw,N,ShapiroWilkPValue,ShapiroWilkPText,ShapiroWilkCallP_5E-2
1,2,0.7380468157991121,888,1e-06,,p
2,13,0.8639321777575533,45,0.01,<0.01,p
3,7,0.7192490933650109,1118,1e-06,,p
4,14,0.9308345215745265,39,0.029586303936316234,,p
5,1,0.7768408108631804,526,1e-06,,p
6,9,0.7646871828003446,845,1e-06,,p
7,12,0.8145813511010308,182,1e-06,,p
8,0,0.8439237666531061,176,1e-06,,p
9,8,0.7397561124692298,1014,1e-06,,p
10,10,0.732851933043747,613,1e-06,,p


4.  D'Agostino and Pearson Test with group-by option.

In [26]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=P;
                          groupby=tot_cust_years;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,tot_cust_years,T,Zkurtosis,Zskew,ChiPValue,ChiPText,ChiCallP_5E-2
1,5,670.6249123670639,13.339258887906915,22.19660074625077,0.0001,<0.0001,p
2,13,6.550323087619043,-0.2341478075898076,2.548626667799725,0.039921993675254674,,p
3,7,688.1595653640874,13.841639526324528,22.283818806196074,0.0001,<0.0001,p
4,0,56.45743848242031,3.748819106192446,6.511819545370309,0.0001,<0.0001,p
5,3,589.343712293948,12.388685407891993,20.877360612832074,0.0001,<0.0001,p
6,11,147.96721301835333,6.177449794326851,10.478851418782714,0.0001,<0.0001,p
7,12,56.86330990567802,3.6355690358930888,6.606507980085502,0.0001,<0.0001,p
8,14,3.328860179628917,-0.6811192150956615,1.6926124170808823,0.20446845179761383,,a
9,1,244.19995139460283,8.296957017858565,13.242373489613195,0.0001,<0.0001,p
10,9,439.9169174138988,11.233736234248122,17.71214520133684,0.0001,<0.0001,p


5.  Smirnov test with group-by option.

In [27]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          columns=female_ind;
                          teststyle=S;
                          groupby=tot_cust_years;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,tot_cust_years,M,N,D,SmirnovPValue,SmirnovPText,SmirnovCallP_5E-2
1,8,450,564,1.3487542965939632,0.005,<0.005,p
2,11,158,216,1.2858397724698776,0.005,<0.005,p
3,12,77,105,1.3420316985107306,0.005,<0.005,p
4,0,74,102,1.3138514694761927,0.005,<0.005,p
5,3,495,665,1.374792997808136,0.005,<0.005,p
6,9,374,471,1.3686535531467623,0.005,<0.005,p
7,4,503,634,1.3763702848743815,0.005,<0.005,p
8,14,16,23,1.371671115432294,0.000101,,p
9,1,226,300,1.3719868864800961,0.005,<0.005,p
10,10,263,350,1.3713273499382055,0.005,<0.005,p


6.  Kolmogorov-Smirnov test that stores its results in an output table val_ks_out1 and does not contain a group-by option.

In [28]:
call ${VALDB}.td_analyze('KSTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=KS; 
                          statsdatabase=${VALDB};
                          outputdatabase=${QLID};
                          outputtablename=_val_ks_out1');

Success: 0 rows affected

In [29]:
SELECT * FROM ${QLID}._val_ks_out1;

,Klm,M,KlmPValue,KlmPText,KlmCallP_5E-2
1,0.15818501100000001,10458,0.01,<0.01,p


---

### Parametric Tests

#### Purpose

Parametric tests make assumptions about the data, such as the observations being normally distributed.  This can be verified with a test of normality prior to executing a parametric test.  Both T-Tests and F-Tests are provided.  T-Tests may be either paired or unpaired, while the unpaired T-Tests may be with or without an indicator variable.  

F-Tests may be 1-way, 2-way or 3-way.  2-way tests may have equal or unequal cell counts (distinct column values), while the 3-way test must have equal cell counts.  A 1-way test has 1 independent input column, a 2-way test has 2 independent columns and a 3-way test has 3 independent columns in addition to a dependent “column of interest”.

#### Required Parameters

- **database**

    The database containing the table to analyze.

- **parametrictest**

    The parametrictest parameter:
    - Is required
    - Must be the first parameter
    - Is always enclosed in single quotes


- **tablename**

    The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.

#### Optional Parameters

- **columnofinterest**

    The column representing the dependent variable in an F-Test.

- **columns**

    The columns representing independent variables to be analyzed in a F-Test N-Way with Equal Cell Counts analysis.  There may be 1, 2 or 3 columns listed in this parameter.  If 2 or 3 columns, the number of distinct values for each combination of column values should be the same.

- **equalvariance**

    An option available with the T-Test where, if true, the variance of the two samples (columns) is assumed to be equal.  The default assumption is that the variances are not equal.

- **fallback**

    When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.

- **firstcolumn**

    For a T-Test, the column representing the first variable to analyze. It must reside in the table indicated by the tablename parameter.  For an F-Test, the column representing the first independent variable in the analysis.

- **firstcolumnvalues**

    This is a required option for a 2-way F-Test with Unequal Cell Counts.  It is a list of the firstcolumn values to be included in the analysis.

- **gensqlonly**

    For a T-Test analysis, when gensqlonly is true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.  This option is not available for an F-Test analysis.

- **groupby**

    One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.

- **multiset**

    When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.

- **outputdatabase**

    Specifies the name of the database to contain the analysis results table.

- **outputtablename**

    Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.

- **overwrite**

    When overwrite is set to true or not set, the output table is dropped before creating a new one.

- **paired**

    An option of the T-Test, which when set to true, indicates that the first and second column values are matched with each other and the mean difference is analyzed.  The paired option defaults to false.

- **secondcolumn**

    For a T-Test, the column representing the second variable to analyze. It must reside in the table indicated by the tablename parameter.  If the withindicator option is set to true, the second column is used to define two analysis categories, one where the second column is negative or zero, and another where the second column is positive.  For an F-Test, the column representing the second independent variable in the analysis.

- **secondcolumnvalues**

    This is a required option for a 2-way F-Test with Unequal Cell Counts.  It is a list of the secondcolumn values to be included in the analysis.

- **statsdatabase**

    This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.

- **teststyle**

    The valid values are t, fnway and f2way, as outlined below. The default is t for T-Test.
    
    - T-Test paired, unpaired or unpaired with indicator variable (second column)
    - F-Test N-Way with Equal Cell Counts (1, 2 or 3 columns with same number of distinct values)
    - F-Test 2-Way with Unequal Cell Counts (2 columns with possibly different numbers of distinct values)


- **thresholdprobability**

    You may optionally change the threshold probability or allow it to default to 0.05.  Below this  "alpha" probability the null hypothesis is rejected.

- **withindicator**

    This option may be used with an un-paired T-Test, that is when teststyle=t and paired=false.  If the withindicator option is set to true, the second column is used to indicate there are two analysis categories, one for the case where the second column is negative or zero, and another when the second column is positive.

1.  T-Test producing an unpersisted result set with default values for teststyle (t) and  thresholdprobabilty (.05).  

In [30]:
call ${VALDB}.td_analyze('ParametricTest', 
                         'database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=cc_avg_bal;
                          secondcolumn=sv_avg_bal;
                          paired=true;
                          equalvariance=true;
                          groupby=tot_age,female_ind;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,tot_age,female_ind,D_F,TTestPValue,T,TTestCallP_0.05
1,85,1,18,0.4807706,-0.715463172636989,a
2,93,0,3,0,,
3,77,0,7,0.2252156,1.3272042858038389,a
4,76,1,24,0.7818767,0.27575427059614926,a
5,72,1,28,0.2042013,-1.2957723582343272,a
6,93,1,7,0.0856193,2.00449476653439,a
7,64,1,46,0.0063499,-2.8563487358795143,n
8,50,1,107,0.0016447,-3.2344818472272943,n
9,47,0,54,0.0053865,-2.9000459446649285,n
10,89,1,15,0.0142988,2.7683360374460215,p


2.  One-Way F-Test producing an unpersisted result set.

In [31]:
call ${VALDB}.td_analyze('ParametricTest',
                         'teststyle=fnway;
                          database=${XSPDB};
                          tablename=Customer;
                          columnofinterest=income;
                          columns=years_with_bank;
                          groupby=gender;
                          thresholdprobability=0.01;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,gender,DF,DFErr,F,FPValue,FPText,FCallP_0.01
1,F,14,5837,7.339460843404131,0.001,<0.001,p
2,M,14,4591,4.946361887968074,0.001,<0.001,p


3.  2-way F-Test with Unequal Cell Counts producing an unpersisted result set.

In [32]:
call ${VALDB}.td_analyze('ParametricTest',
                         'teststyle=f2way;
                          database=${XSPDB};
                          tablename=Customer;
                          columnofinterest=income;
                          firstcolumn=years_with_bank;
                          secondcolumn=gender;
                          thresholdprobability=0.05;
                          firstcolumnvalues=0,1,2,3,4,5,6,7;
                          secondcolumnvalues=M,F;
                          statsdatabase=${VALDB};');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,DF,Fmodel,DFErr,DF_1,F1,DF_2,F2,DF_12,F12,Fmodel_PValue,Fmodel_PText,Fmodel_CallP_0.05,F1_PValue,F1_PText,F1_CallP_0.05,F2_PValue,F2_PText,F2_CallP_0.05,F12_PValue,F12_PText,F12_CallP_0.05
1,15,9.710997531996655,7330,7,11.561359030335888,1,51.83852316974105,7,1.8424180854082204,0.001,<0.001,p,0.001,<0.001,p,0.001,<0.001,p,0.07854148392066758,,a


---

### Rank Tests

#### Purpose

Statistical tests of this type calculate statistics based on the rank of variables rather than variable values.  In general, data that are ranked and ordinal may be analyzed by these tests.  Within some restraints, either numeric or non-numeric data may be analyzed.  Supported rank tests include the following:

- Mann-Whitney/Kruskal-Wallis Test
- Wilcoxon Signed Ranks Test
- Friedman Test with Kendall’s Coefficient of Concordance & Spearmans’ Rho

The choice between the Mann-Whitney and Kruskal-Wallis tests is made automatically, looking at the number of distinct values of the independent variable.  A variation of the Mann-Whitney test considers each requested variable individually, rather than combined, performing a series of independent tests.

#### Required Parameters

- **database**

    The database containing the table to analyze.

- **ranktest**

    The ranktest parameter:
    - Is required
    - Must be the first parameter
    - Is always enclosed in single quotes
    

- **tablename**

    The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.

#### Optional Parameters

- **blockcolumn**

    Used only by the Friedman test, the column that represents blocks. They must reside in the table indicated by the tablename parameter.  Note that when pairing treatment and block column values, a division by zero error may occur if unequal cell counts are found.

- **columnofinterest**

    Used only by the Mann-Whitney and Friedman tests, the column specified with this parameter indicates the dependent variable. If non-numeric, it will be ranked alphanumerically.  It must reside in the table indicated by the tablename parameter.

- **columns**

    Used only by the Mann-Whitney test, the columns specified with this parameter represent the independent variables. They must reside in the table indicated by the tablename parameter.

- **fallback**

    When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.

- **firstcolumn**

    Used only by the Wilcoxon test, the column that represents the first sample variable. It must reside in the table indicated by the tablename parameter.

- **gensqlonly**

    When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.  This parameter is not allowed when using the Mann-Whitney test.

- **groupby**

    One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.

- **includezero**

    Used only by the Wilcoxon test.  Ordinarily, the Wilcoxon test discards cases with zero differences.  This option, when set to true, includes these cases with the positive count. 

- **independent**

    Used only by the Mann-Whitney test, its selection by setting independent=true, indicates that a variation of the Mann-Whitney test should be performed, considering each requested variable individually, rather than in combination, performing a series of independent tests.

- **multiset**

    When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.

- **outputdatabase**

    Specifies the name of the database to contain the analysis results table.

- **outputtablename**

    Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.

- **overwrite**

    When overwrite is set to true or not set, the output table is dropped before creating a new one.

- **secondcolumn**

    Used only by the Wilcoxon test, the column that represents the second sample variable. It must reside in the table indicated by the tablename parameter.  Note that when pairing treatment and block column values, a division by zero error may occur if unequal cell counts are found.

- **singletail**

    Used only by the Mann-Whitney and Wilcoxon tests.  If the Mann-Whitney test becomes a Kruskall-Wallis test, the singletail option is invalid.  By default, a two-tailed test is performed.

- **statsdatabase**

    This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.

- **teststyle**

    The valid values are mw, friedman and wilcoxon, representing a Mann-Whitney, Friedman or Wilcoxon test.  If not specified, the default test style is Mann-Whitney (mw).

- **thresholdprobability**

    You may optionally change the threshold probability or allow it to default to 0.05.  Below this "alpha" probability the null hypothesis is rejected.

- **treatmentcolumn**

    Used only by the Friedman test, the column that represents the independent categorical variable. They must reside in the table indicated by the tablename parameter.

1.  Mann-Whitney test with a threshold probability of 0.01.

In [33]:
call ${VALDB}.td_analyze('ranktest',
                         'teststyle=mw;
                          statsdatabase=${VALDB};
                          database=${XSPDB};
                          tablename=Customer;
                          columnofinterest=income;
                          columns=gender;
                          groupby=years_with_bank;
                          thresholdprobability=0.01;');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,years_with_bank,Z,MannWhitneyPValue,MannWhitneyCallP_1E-2
1,12,-1.5058426157251128,0.132067226,a
2,3,-1.2277790991000173,0.219447096,a
3,4,-2.1776290109979204,0.029405932,a
4,2,0.2632551342578662,0.792550692,a
5,11,-1.329063927064864,0.183848096,a
6,10,-3.2379133262969324,0.001203832,n
7,7,-3.9525279885398006,7.7211e-05,n
8,13,-0.8617464449585489,0.388687424,a
9,14,-0.49966021063014066,0.617075066,a
10,8,-3.508754308300577,0.00044988199999999997,n


2.  A set of Mann-Whitney independent tests.  The threshold probability assumes the default value of 0.05.

In [34]:
call ${VALDB}.td_analyze('ranktest',
                         'teststyle=mw;
                          statsdatabase=${VALDB};
                          independent=true;
                          database=${QLID};
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          columns=female_ind,cc_acct_ind,ck_acct_ind,sv_acct_ind; ');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,_twm_independent_variable,Z,MannWhitneyPValue,MannWhitneyCallP_5E-2
1,female_ind,7.771020590981125,6e-07,p
2,sv_acct_ind,9.377586778932272,6e-07,p
3,cc_acct_ind,-19.115808047105958,6e-07,n
4,ck_acct_ind,-16.06418788422782,6e-07,n


3.  Wilcoxon Test - the threshold probability assumes the default value of 0.05.

In [35]:
call ${VALDB}.td_analyze('ranktest',
                         'teststyle=wilcoxon;
                          statsdatabase=${VALDB};
                          database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumn=ck_avg_bal;
                          secondcolumn=sv_avg_bal;
                          groupby=tot_cust_years; ');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,tot_cust_years,N,Z_,WilcoxonPValue,WilcoxonCallP_5E-2
1,11,315,-8.214871312499245,6e-07,n
2,13,39,-2.6793596002730693,0.007384332,n
3,7,918,-11.640686617914307,6e-07,n
4,5,1010,-16.95474369795407,6e-07,n
5,9,681,-9.077457386798521,6e-07,n
6,4,977,-18.779568835263486,6e-07,n
7,0,150,-4.790039389110829,1.669944e-06,n
8,14,33,-2.206676899498888,0.027313964,n
9,3,1003,-17.354268985435144,6e-07,n
10,1,454,-8.994275181286014,6e-07,n


4.  A Friedman Test using a specially prepared input table called VAL_Friedman_WorkTable.  To build the VALFriedmanWorkTable, the following SQL may be used.  Note that the value ‘18’ is the smallest count of value combinations in the gender and marital_status columns in the Customer table.  The value '18'  is determined using the following query:

In [36]:
SELECT 
     MIN("T0"."_val_N") AS "smallest_count"
FROM 
( SELECT 
     "S0"."marital_status" AS "marital_status"
    ,"S0"."gender" AS "gender"
    ,COUNT(*) AS "_val_N"
FROM "${XSPDB}"."Customer" AS "S0"
GROUP BY "S0"."marital_status", "S0"."gender"
) AS "T0"
;

,smallest_count
1,252


In [37]:
CREATE SET TABLE "${QLID}"."VAL_Friedman_Worktable" AS (
SELECT
    "cust_id"
    ,"gender"
    ,"marital_status"
    ,"income"
    ,"ckacct"
    ,"svacct"
    ,SAMPLEID AS "xSampleId"
FROM "TRNG_XSP"."twm_customer_analysis"
SAMPLE 
    WHEN gender='f' and marital_status='1' THEN 18
    WHEN gender='f' and marital_status='2' THEN 18
    WHEN gender='f' and marital_status='3' THEN 18
    WHEN gender='f' and marital_status='4' THEN 18
    WHEN gender='m' and marital_status='1' THEN 18
    WHEN gender='m' and marital_status='2' THEN 18
    WHEN gender='m' and marital_status='3' THEN 18
    WHEN gender='m' and marital_status='4' THEN 18
END
) WITH DATA PRIMARY INDEX("cust_id");

Success: 0 rows affected

In [38]:
call ${VALDB}.td_analyze('ranktest',
                         'teststyle=friedman;
                          statsdatabase=${VALDB};
                          database=${QLID};
                          tablename=VAL_Friedman_Worktable;
                          columnofinterest=income;
                          treatmentcolumn=gender;
                          blockcolumn=marital_status;');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,Kendalls_W,Average_Spearmans_Rho,DF_1,DF_2,F,FriedmanPValue,FriedmanPText,FriedmanCallP_5E-2
1,0.7617681316974686,0.7724972603129259,1,71,227.02897700418848,0.001,<0.001,p


---

### Chi Square Tests

#### Purpose

Statistical tests of this type are based on a matrix of frequencies or counts.  A frequency pattern that is non-random is sought in the matrix.  Supported tests of this type include the following:

- Chi Square Test

    Besides a Chi Square value, other measures are computed in a Chi Square Test, including a Phi Coefficient,  Cramer’s V, Likelihood Ratio Chi Square, Continuity-Adjusted Chi Square and Contingency Coefficient.

- Median Test

    A Median Test is a variation of Chi Square Test wherein samples are tested to see if their populations have the same median value.

#### Required Parameters

- **chisquaretest**

    The chisquaretest parameter:
    
    - Is required
    - Must be the first parameter
    - Is always enclosed in single quotes


- **database**

    The database containing the table to analyze.

- **tablename**

    The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.

#### Optional Parameters

- **columnofinterest**

    Used only by the Median Test, the column specified with this parameter indicates the dependent variable (numeric).  It must reside in the table indicated by the tablename parameter.

- **columns**

    Used only by the Median Test, the columns specified with this parameter represent the independent variables (categorical). They must reside in the table indicated by the tablename parameter.

- **fallback**

    When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.

- **firstcolumns**

    Used only by the Chi Square Test, one or more columns representing the first of variable pairs for analysis. They must reside in the table indicated by the tablename parameter.  Note that the number of combinations of firstcolumns and secondcolumns may not exceed 100.  Note also that if the product of the number distinct values in these column pairs exceeds 2000, the analysis of that combination is skipped (but not if gensqlonly is true).

- **gensqlonly**

    When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.

- **groupby**

    Available only in the Median Test, one or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.

- **multiset**

    When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.

- **outputdatabase**

    Specifies the name of the database to contain the analysis results table.

- **outputtablename**

    Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.

- **overwrite**

    When overwrite is set to true or not set, the output table is dropped before creating a new one.

- **secondcolumns**

    Used only by the Chi Square Test, one or more columns representing the second of variable pairs for analysis. They must reside in the table indicated by the tablename parameter.  Note that the number of combinations of firstcolumns and secondcolumns may not exceed 100.  Note also that if the product of the number distinct values in these column pairs exceeds 2000, the analysis of that combination is skipped (but not if gensqlonly is true).

- **statsdatabase**

    This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.

- **teststyle**

    The valid values are chisq and median, representing a Chi Square or Median test.  If not specified, the default test style is Chi Square (chisq).

- **thresholdprobability**

    You may optionally change the threshold probability or allow it to default to 0.05.  Below this "alpha" probability the null hypothesis is rejected.

1.  Chi Square Test with non-persisted results.

In [39]:
call ${VALDB}.td_analyze('chisquaretest',
                         'statsdatabase=${VALDB};
                          teststyle=chisq;
                          database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumns=female_ind,single_ind;
                          secondcolumns=sv_acct_ind,cc_acct_ind,ck_acct_ind;');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,column1,column2,Chisq,DF,Z,CramersV,PhiCoeff,LlhChiSq,ContAdjChiSq,ContinCoeff,ChiPValue,ChiPText,ChiCallP_5E-2
1,female_ind,ck_acct_ind,51.963291143618186,1,6.266072413236709,0.07048943119690093,0.07048943119690093,51.79027403137195,51.65502985345838,0.07031495858282666,0.0001,<0.0001,p
2,single_ind,sv_acct_ind,111.33800714811451,1,8.555265794438732,0.10318044653217087,0.10318044653217087,112.15138339109859,110.90737820419025,0.10263555340590276,0.0001,<0.0001,p
3,female_ind,sv_acct_ind,73.52013882165667,1,7.236808261366889,0.08384532268145403,0.08384532268145403,73.48997707707434,73.17998066361037,0.08355214963436527,0.0001,<0.0001,p
4,single_ind,cc_acct_ind,21.541917084755497,1,4.252562129760672,0.04538557648792909,0.04538557648792909,21.461624947837834,21.347967794596414,0.04533890482563551,0.0001,<0.0001,p
5,single_ind,ck_acct_ind,3.722261430607426,1,1.6376557333933495,0.018865969038997184,0.018865969038997184,3.7119507979566606,3.6377618158186986,0.018862612501963423,0.055246604425102215,,a
6,female_ind,cc_acct_ind,84.45466569698048,1,7.657179013939365,0.08986436396365856,0.08986436396365856,84.28739385913224,84.08086832585147,0.08950369247502081,0.0001,<0.0001,p


2.  Chi Square Test with results persisted in an output table.

In [40]:
call ${VALDB}.td_analyze('chisquaretest',
                         'statsdatabase=${VALDB};
                          teststyle=chisq;
                          database=${QLID};
                          tablename=VAL_ADS;
                          firstcolumns=female_ind,single_ind;
                          secondcolumns=sv_acct_ind,cc_acct_ind,ck_acct_ind;
                          outputdatabase=${QLID};
                          outputtablename=_val_chisquare_out;');

Success: 0 rows affected

In [41]:
SELECT * FROM ${QLID}._val_chisquare_out ORDER BY 1;

,column1,column2,Chisq,DF,Z,CramersV,PhiCoeff,LlhChiSq,ContAdjChiSq,ContinCoeff,ChiPValue,ChiPText,ChiCallP_5E-2
1,female_ind,sv_acct_ind,73.52013882165667,1,7.236808261366889,0.08384532268145403,0.08384532268145403,73.48997707707434,73.17998066361035,0.08355214963436527,0.0001,<0.0001,p
2,female_ind,ck_acct_ind,51.96329114361818,1,6.266072413236709,0.07048943119690093,0.07048943119690093,51.79027403137195,51.65502985345838,0.07031495858282666,0.0001,<0.0001,p
3,female_ind,cc_acct_ind,84.45466569698048,1,7.657179013939365,0.08986436396365856,0.08986436396365856,84.28739385913224,84.08086832585147,0.08950369247502081,0.0001,<0.0001,p
4,single_ind,cc_acct_ind,21.5419170847555,1,4.252562129760672,0.04538557648792909,0.04538557648792909,21.461624947837834,21.347967794596414,0.04533890482563551,0.0001,<0.0001,p
5,single_ind,sv_acct_ind,111.33800714811453,1,8.555265794438734,0.10318044653217087,0.10318044653217087,112.15138339109859,110.90737820419025,0.10263555340590277,0.0001,<0.0001,p
6,single_ind,ck_acct_ind,3.722261430607426,1,1.6376557333933495,0.018865969038997184,0.018865969038997184,3.7119507979566606,3.6377618158186986,0.018862612501963423,0.055246604425102215,,a


3.  Median Test with groupby column years_with_bank, thresholdprobability set to 0.01 and results returned in a result set.  

In [42]:
call ${VALDB}.td_analyze('chisquaretest', 
                         'statsdatabase=${VALDB};
                          teststyle=median;
                          database=${XSPDB};
                          tablename=Customer;
                          columnofinterest=income;
                          columns=marital_status;
                          groupby=years_with_bank;
                          thresholdprobability=0.01;');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,years_with_bank,ChiSq,DF,MedianPValue,MedianPText,MedianCallP_1E-2
1,3,100.0969510011295,3,0.0001,<0.0001,p
2,5,98.98244733043795,3,0.0001,<0.0001,p
3,0,16.604904354904356,3,0.0008843323011702343,,p
4,14,2.233403110047847,3,0.25,>0.25,a
5,12,9.167059563099166,3,0.027956035435991015,,a
6,4,77.54838819017066,3,0.0001,<0.0001,p
7,7,74.72117984547474,3,0.0001,<0.0001,p
8,2,111.94131226907886,3,0.0001,<0.0001,p
9,13,2.8559997422237497,3,0.25,>0.25,a
10,9,23.73665898145068,3,0.0001,<0.0001,p


---

Since we are in a beta phase, the following command can be used to determine the specific release you are running:

In [43]:
call ${VALDB}.td_analyze('version','');

Success: 0 rows affected

WARNING: [Teradata Database] [Warning 3212] The stored procedure returned one or more result sets.

,version
1,VAL In-DB 2.1.0.0


In [44]:
DROP TABLE "${QLID}"."VAL_Friedman_Worktable";

Success: 21 rows affected

In [45]:
DROP TABLE "${QLID}"."VAL_ADS";

Success: 42 rows affected

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2022 Teradata. All Rights Reserved</footer>